In [180]:
import csv
import json
import twitter
import pandas as pd
from textblob import TextBlob
#from textblob_fr import PatternTagger, PatternAnalyzer
import matplotlib.pyplot as plt
import numpy as np
import math
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

consumer_key = "BuD5m4LWu89up33IGUOIhINuY"
consumer_secret = "oKG5Ip88Vjv1PDfylElVLYtVLf6dADGg7zqAVB5ZEYnw1v27Bo"
access_token_key = "955591406067871744-LgQ25aRMPJ81LGdgzmXNPUO6VHfl52K"
access_token_secret = "x41eaT0zG6klEbBTbD815B2uDEfFCtJggafQnZfCIsu3Y"


In [181]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [182]:
quebecMedia = ["@tvanouvelles","@JdeMontreal","@LP_LaPresse","@RadioCanadaInfo","@JdeQuebec","@vicequebec",
"@infomantv","@salutbonjour","@CBCMontreal","@TVASports","@RDSca","@icimontreal","@meteomedia","@CTVMontreal",
"@Global_Montreal","@telequebec","@LaTribune","@_URBANIA","@HuffPostQuebec","@LeDevoir","@mtlgazette","@iciquebec",
"@cyblesoleil","@Lactualite","@coupdepouce_mag","@la_lesaffaires","@journalmetro","@icircpremiere", "@RNCNouvellesAT"]

francemedia = ["@Le_Figaro","@lemondefr","@canardenchaine","@Qofficiel", "@20hFrance2", "@TF1LeJT", "@LCI", "@BFMTV", 
              "@CNEWS", "@libe", "@lequipe", "@le_Parisien", "@20Minutes", "@lobs" ,"@LesEchos","@TV5MONDE", 
              "@Le_Soir3", "@ARTEjournal", "@LaTribune", "@franceinfo", "@FRANCE24","@rtlinfo","@Europe1",
              "@RMCinfo", "@franceinter","@francebleu","@Charlie_Hebdo_", "@LEXPRESS", "@LePoint", "@VICEfr", "@radiofrance", 
              "@mdiplo", "@Mediapart", "@RTLFrance", "@franceinfo", "@lejdd", "@RTenfrançais"]



columns_to_keep = ["created_at","display_text_range","entities","favorite_count","full_text","retweet_count","sentiment","searchTerm"]

In [183]:
len(francemedia)

37

In [184]:
t = twitter.Api(consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token_key=access_token_key,
    access_token_secret=access_token_secret,
    tweet_mode="extended"
    )

In [193]:
def tweetsSearch(searchTerm="quebecMedia",lang=None,since="2019-03-20",until="2019-03-26"):
    tweets = t.GetSearch(term = searchTerm,count = 100,return_json = True,lang=lang,since=since, until=until, result_type ="mixed")
    return tweets

In [1]:
def weekScrap(searchTerm):
    date = "2019-03-2"
    for i in range(1,6):
        tweets = tweetsSearch(searchTerm=searchTerm,lang='fr',since=date+str(i),until=date+str(i+1))
        if(i==1):
            pd_tweets = pd.DataFrame(tweets["statuses"])
        else:
            pd_tweets = pd.concat([pd_tweets,pd.DataFrame(tweets["statuses"])])
    return pd_tweets

In [195]:
pd_tweets = weekScrap(tweetsSearch)

In [196]:
def getRTStatusId(row):
    try:
        if(math.isnan(row["retweeted_status"])):
            return row["id"]
    except:
        return row["retweeted_status"]["id"]

def replaceRTFullText(row):
    try:
        if(math.isnan(row["retweeted_status"])):
            return row["full_text"]
    except:
        return row['retweeted_status']["full_text"]

    
def rmDuplicatedRT(df):
    df['original_status_id'] = df.apply(lambda row: getRTStatusId(row),axis=1)    
    df['full_text'] = df.apply(lambda row: replaceRTFullText(row),axis=1)
    return df.drop_duplicates(subset="original_status_id")

In [197]:
print(len(pd_tweets["full_text"].values))
pd_tweets = rmDuplicatedRT(pd_tweets)
print(len(pd_tweets["full_text"].values))

KeyError: 'full_text'

In [198]:
def sentiment_score(sentence):
    fr_blob = TextBlob(sentence)
    try:
        #translate sentence to english
        sentence_eng = fr_blob.translate(to='eng')
    except:
        sentence_eng = sentence
    #compute the polarity score -1 = negative, 1 = positive (continuous values)
    snt = analyser.polarity_scores(sentence_eng)
    return snt["compound"]

In [199]:
def cleanScrap(searchterm):
    #Scrap tweets
    pd_tweets = weekScrap(searchterm)
    
    #Keep one copy of retweeted tweets
    pd_tweets = rmDuplicatedRT(pd_tweets)
    
    #compute the sentiment of tweets
    pd_tweets["sentiment"] = pd_tweets.apply(lambda row: sentiment_score(row["full_text"]),axis=1)
    
    pd_tweets["searchTerm"] = searchterm
    
    return pd_tweets

def fullScrap(searchterm_list):
    init = False
    
    for term in searchterm_list:
        if(not init):
            df_all_tweet = cleanScrap(term)
            print(len(df_all_tweet["full_text"].values))
            init=True
        else:
            df_all_tweet = pd.concat([df_all_tweet,cleanScrap(term)])
            print(len(df_all_tweet["full_text"].values))

    return df_all_tweet

def saveToCSV(df,filePath, sep="|"):
    df[columns_to_keep].to_csv(filePath,sep="|", encoding='utf-8')

In [200]:
pd_tweets = cleanScrap("quebecMedia")
pd_tweets.head()

ValueError: Wrong number of items passed 0, placement implies 1

In [ ]:
df_quebec = fullScrap(quebecMedia)
saveToCSV(df_quebec,"QuebecMedia", "sort = True")

In [ ]:
df_monde = fullScrap(["@LeMonde"])

In [ ]:
data = [go.Histogram(x=pd_tweets["sentiment"])]
iplot(data)

In [ ]:
print(len(pd_tweets))
pd_tweets.columns

In [ ]:
from nbconvert import HTMLExporter
import codecs
import nbformat
exporter = HTMLExporter()
output_notebook = nbformat.read('TwitterScrapping.ipynb', as_version=4)
output, resources = exporter.from_notebook_node(output_notebook)
codecs.open('TwitterScrapping.html', 'w', encoding='utf-8').write(output)